# PPD: OpenMP / variáveis compartilhadas e privadas

Hélio - DC/UFSCar - 2023

# Sobre variáveis compartilhadas e privadas

Considerando uma região paralela com múltiplas *threads* dentro do programa, é preciso pensar sobre o comportamento das variáveis. Por padrão, salvo se tratado explicitamente, todas as variáveis do programa (globais e da função *main*) são naturalmente **compartilhadas** pelas *threads* de uma região paralela.

Vejamos um exemplo de programa em C, que ilustra o uso de variáveis globais do programa e de variáveis definidas dentro da função *main*, sendo manipuladas dentro da região paralela, por todas as *threads*.
```
 1  ...
 2  int global;
 3  ...
 4  
 5  void f()
 6  {
 7    int val; ...    // local, alocada na pilha de cada thread que chamar esta função...
 8    ...
 9  }
10
11  int main()
12  {
13    int i, num, sum=0;
14    ...
15    #pragma omp parallel private( ... )
16    {
17      ...
18      if (global > ...) ;    // acesso de leitura à variável global
19      ...
20      num = ...;             // acesso de escrita na variável num, compartilhada pelas threads do time
21      ...
22      #pragma omp for private( ...  )
23      for ( i = ...; i < global ; ... ) {    // leitura de global, alteração de i: compartilhada?
24        ...
25        sum = sum + ...;     // alteração de sum. Qual é o valor inicial das cópias? Como ajustar o valor das cópias ao fim do loop?
25      }
26    }
27    ...
28    printf("Soma: %d\n", sum);
28  }
```


# Visibilidade das variáveis

Examinando o programa acima, primeiro, é preciso ter claro quais variáveis do programa serão acessíveis pelas *threads* que forem criadas.

A variável **global**, por ter sido declarada fora do escopo de qualquer função do programa, será alocada no BSS, ou seja, numa área de variáveis globais estáticas do programa e, portanto, acessível em qualquer parte do código e **compartilhada**.

As variáveis ***i***, ***num*** e ***sum*** foram declaradas dentro da função *main* e portanto serão **alocadas na pilha** da *thread* associada à função *main*. Como o compilador com suporte a OpenMP enxerga essas variáveis no bloco de código que cria a região paralela, ele consegue resolver as referências a essas variáveis, de forma que elas, ou cópias delas, serão acessíveis pelas *threads* do time da região paralela.

Já a variável ***val***, declarada no escopo da *função f()*, só vai ser alocada sob demanda, **na pilha de cada thread** que invocar essa função. Assim, ela não é visível pelas demais *threads* e não tem sentido considerarmos seu possível compartilhamento (salvo se ela fosse declarada como estática (*static*), o que a colocaria na mesma condição da variável global, alocaca no BSS).


(@h: to do: variável ponteiro + malloc())


# Tipo de acesso às variáveis compartilhadas

Outra questão a considerar sobre o uso de variáveis em regiões paralelas é o tipo de acesso a elas. A variável **global**, por exemplo, é acessada 2 vezes dentro da região paralela. Em ambos os casos, linhas 18 e 23, os acessos a ela são apenas para **leitura** de seu valor. Não há problema, portanto, que ela seja compartilhada, ou mesmo que seu conteúdo seja acessado por várias *threads* ao mesmo tempo, pois não há risco de alguma *thread* ler um valor inconsistente (*half-written*).

Vejamos agora o acesso à variável ***i***, na linha 23. Trata-se de um *for paralelo*, em que as iterações do *loop* serão divididas entre as *threads* do time. Para que cada *thread* manipule o seu sub-conjunto de iterações, é claro que cada uma deve **ter sua própria cópia** desta variável. Cada *thread* vai atribuir à sua instância da variável ***i*** o valor inicial da iteração e incrementá-lo a cada rodada, comparando-o com o valor limite **local** (das iterações desta *thread*).

Já sabemos, contudo, que **o compilador** com suporte a OpenMP, automaticamente, faz com que a variável de controle de um *parallel for* seja **privada** para cada *thread*, e vai tratar também de ajustar o código de cada *thread* para a atribuição do valor inicial apropriado e determinar o limite e o passo para cada iteração.

Vejamos agora o acesso à variável **num**, na linha 20: num = ...  . Trata-se de um acesso de **escrita** numa variável acessível por todas as *threads* do time. A decisão se essa variável deve ser **compartilhada** (***shared*** ou **privada** (***private***) depende de seu **uso** dentro do código da região paralela.

Se a questão é apenas o fato de que cada *thread* deve ter sua própria cópia desta variável, sem que o valor de dada uma delas tenha que ser visível pelas demais *threads*, basta fazer com que essa variável seja privada. Isso é feito através da ***cláusula private***, usada na diretiva de criação da região paralela.

***15    #pragma omp parallel private( num )***

Por outro lado, se ela precisa ser compartilhada, para que todas as *threads* do time tenham acesso aos valores que ela armazena, as operações de escrita sobre ela talvez tenham que ser **protegidas** por algum **mecanismo de sincronização**.

E a variável ***sum***? Valem as mesmas considerações para a variável ***num***, mas há outros aspectos sobre seu uso que trataremos posteriormente...



# Exclusão mútua e operação atômica

Uma das formas de garantir que a escrita a uma variável compartilhada dentro de uma região paralela ocorra **com exclusão mútua** entre as *threads* do time é usar um mecanismo de **região crítica**, como ilustrado a seguir, com a diretiva ***critical***:
```
#pragma omp critical
 num = ...;
```
Usada dentro de uma região paralela, a diretiva *critical* faz com que o bloco de código a seguir seja executado com *exclusão mútua*, por **uma *thread* do time de cada vez**. É claro, contudo, que esse mecanismo pode ter impacto no desempenho da aplicação, ao reduzir (inibir) o paralelismo neste trecho de código.

Para alguns casos específicos de alteração de variável compartilhada, há outra forma de garantir o acesso com exclusão mútua. Isso é feito com a **diretiva *atomic*** ([man](https://www.openmp.org/spec-html/5.0/openmpsu95.html)), como ilustrado a seguir. Usada dentro de uma região paralela, essa diretiva determina que uma posição de memória específica deve ser **lida e/ou atualizada de maneira atômica**. Quando possível, esse tipo de acesso é traduzido pelo compilador em instruções do tipo *fetch-and-add*, impedindo múltiplas *threads* de escrever nessa localização ou lê-la simultaneamente.
```
#pragma omp atomic write
 num = ...;
 ```
Há restrições sobre os formatos que podem ser usados na expressão executada de forma atômica, variando para leitura, escrita e atualização, por exemplo.


# Variáveis privadas: atribuição inicial e propagação de valores

Por fim, um outro aspecto a considerar no uso de variáveis privadas é a relação dessas variáveis com as variáveis compartilhadas de mesmo nome, às quais estão relacionadas.

Em alguns casos, é preciso que, na criação do time de *threads*, cada cópia privada de uma variável seja **inicializada com o valor atual da variável** comum à qual estão relacionadas. Isso pode ser feito com a **cláusula *firstprivate***.
```
15    #pragma omp parallel firstprivate( num )
```
Neste caso, cada *thread* terá uma **cópia** da variável ***num***, sendo que todas elas terão como valor inicial o valor atual da variável ***num*** **na *master thread*** (função *main*, neste caso).

Outra forma de atribuir um valor inicial para as cópias das variáveis privadas das *threads* é através da **cláusula *copyin***. Da mesma maneira que *fistprivate*, o valor da variável da *thread master* (comumente a função main) é copiado para as cópias dessa variável em todas as *threads* do time. Essa cláusula, contudo, **só serve para variáveis globais**, declaradas fora do escopo de qualquer função.
```
22    #pragma omp parallel copyin( global )
```

# Variáveis privadas: retorno do valor da variável associada à última iteração

Considerando ainda o cenário em que um conjunto de iterações é dividido entre múltiplas *threads*, há casos em que a continuação do programa depois da região paralela depende do valor que uma variável compartilhada assume **na última iteração** do *for* compartilhado.

Para esses casos, é possível usar a **cláusula *lastprivate***, que faz com que uma variável seja privada, com cópia para cada *thread*. Além disso, ao final do *loop*,  o valor da variável privada da *thread* que realizar a última iteração seja copiado para a variável de mesmo nome na função *master thread* (*main*).
```
15    #pragma omp for lastprivate( num )
```

Ainda sobre o retorno de valor de cópia de variável privada, é preciso conhecer a cláusula ***reduction***.

A cláusula ***reduction*** pode aparecer quantas vezes forem necessárias em uma diretiva, especificando, em cada uma delas, o operador de redução e o nome de uma variável. Esta variável será tornada automaticamente privada para cada *thread* do time e, ao final da execução do bloco paralelo, os valores dessas cópias serão agregados, usando a operação de redução especificada, e o valor resultante será salvo na variável de mesmo nome na função *master*.

No exemplo a seguir, a variável ***sum*** é tornada privada para cada *thread*, cada cópia é inicializada com 0 e, ao final do bloco paralelo, os valores de todas as cópias são **juntados** através da operação de **soma**. Esse valor agregado pela operação definida na redução é então copiado para a variável correspondente da *master thread* (main).
```
22      #pragma omp for private(num) reduction (+:sum)
```
Há uma série de operações já pré-definidas para a redução, como +, *, -, &, |, ^, && e ||. O valor de inicialização de cada cópia depende do operador selecionado. Como dito, na soma, cada cópia da variável é iniciada com 0 e, para a multiplicação, por exemplo, são inicializadas com 1.

.

Bem, há muito que pensar e tratar com relação a variáveis privadas e compartilhadas, mas espero que esse começo ajude!

Bons estudos,

Hélio

## reduction

Ah, vale saber que a redução pode ocorrer sobre vetores, de forma alinhada elemento a elemento, como ilustrado a seguir.

In [ ]:
%%writefile vec-reduction.c

#include <stdio.h>
#include <string.h>
#include <unistd.h>
#include <stdlib.h>

#define NUM 10

int main()
{
  int sum[NUM];
  int sum2[NUM];

  // zera os vetores
  memset(sum,0,NUM * sizeof(int));
  memset(sum2,0,NUM * sizeof(int));

  for(int i=0; i< NUM;i++) {
    for(int j=0; j< NUM; j++) {
      // usleep(rand()%1000);
      sum[i] += j;
    }
  }
  for(int i=0; i<NUM; i++)
    printf("%d:%d ",i,sum[i]);
  printf("\n");

  // #pragma omp parallel for private(sum2)
  // #pragma omp parallel for lastprivate(sum2)
  #pragma omp parallel for reduction(+:sum2[:NUM])
  for(int i=0; i< NUM;i++) {
    for(int j=0; j< NUM; j++) {
      // usleep(rand()%1000);
      sum2[i] += j;
    }
  }
  for(int i=0; i<NUM; i++)Já a variável val, declarada no escopo da função f(), só vai ser alocada sob demanda, na pilha de cada thread que invocar essa função. Assim, ela não é visível pelas demais threads e não tem sentido considerarmos seu possível compartilhamento (salvo se ela fosse declarada como estática (static), o que a colocaria na mesma condição da variável global, alocaca no BSS).
    printf("%d:%d ",i,sum2[i]);
  printf("\n");

  return 0;
}


Overwriting vec-reduction.c


In [ ]:
! if [ ! vec-reduction -nt vec-reduction.c ]; then gcc -Wall vec-reduction.c -o vec-reduction -fopenmp; fi
! ./vec-reduction

0:45 1:45 2:45 3:45 4:45 5:45 6:45 7:45 8:45 9:45 
0:45 1:45 2:45 3:45 4:45 5:45 6:45 7:45 8:45 9:45 
